In [10]:
import os
import numpy as np
import time
import datetime
import statistics

def startFeature():
    print("Welcome to the Student Management System")

    student_level = input("Please select your student level (U for Undergraduate, G for Graduate, B for Both): ").upper()
    while student_level not in ['U', 'G', 'B']:
        print("Invalid input. Please choose from U, G, or B.")
        student_level = input("Please select your student level (U for Undergraduate, G for Graduate, B for Both): ").upper()

    graduate_level = None
    if student_level in ['G', 'B']:
        graduate_level = input("Please select your graduate level (M for Master, D for Doctorate, B0 for Both): ").upper()
        while graduate_level not in ['M', 'D', 'B0']:
            print("Invalid input. Please choose from M, D, or B0.")
            graduate_level = input("Please select your graduate level (M for Master, D for Doctorate, B0 for Both): ").upper()

    student_id = input("Please enter your 9-digit student ID (e.g. 202006000): ")
    while not student_id.isdigit() or len(student_id) != 9:
        print("Invalid student ID. Please ensure it is exactly 9 digits.")
        student_id = input("Please enter your 9-digit student ID (e.g. 202006000): ")

    print("Processing your selection...")
    time.sleep(3)
    menuFeature(student_level, graduate_level, student_id)

def menuFeature(student_level, graduate_level, student_id):
    session_requests = []  # Keep track of requests made during the session

    while True:
        print("\nStudent Management System Menu")
        print("1. Student Details")
        print("2. Statistics")
        print("3. Transcript based on major courses")
        print("4. Transcript based on minor courses")
        print("5. Full transcript")
        print("6. Previous transcript requests")
        print("7. Select another student")
        print("8. Terminate the system")

        choice = input("Please select an option (1-8): ")

        if choice == '1':
            processRequest("Student Details")
            studentDetails(student_id)
            session_requests.append("Student Details")
        elif choice == '2':
            processRequest("Statistics")
            statisticsFeature(student_id)
            session_requests.append("Statistics")
        elif choice == '3':
            processRequest("Major Transcript")
            transcriptFeature(student_id, "Major")
            session_requests.append("Major Transcript")
        elif choice == '4':
            processRequest("Minor Transcript")
            transcriptFeature(student_id, "Minor")
            session_requests.append("Minor Transcript")
        elif choice == '5':
            processRequest("Full Transcript")
            transcriptFeature(student_id, "Full")
            session_requests.append("Full Transcript")
        elif choice == '6':
            previousRequests(student_id, session_requests)
        elif choice == '7':
            time.sleep(3.5)  # Delay before starting a new feature
            startFeature()
            return  # Exit the menu and restart the program
        elif choice == '8':
            terminateSystem(student_id, session_requests)
            break
        else:
            print("Invalid choice. Please try again.")

def processRequest(request_name):
    print(f"Processing your request for {request_name}...")
    time.sleep(3.5)  # Delay before showing the result

def studentDetails(student_id):
    folder_path = r"C:\\Users\\admin\\JupyterJade"
    file_name = "studentDetails.csv"
    file_path = os.path.join(folder_path, file_name)

    if not os.path.exists(file_path):
        print(f"No file found for Student ID {student_id}.")
        return

    try:
        data = np.loadtxt(file_path, delimiter=',', dtype=str, skiprows=1)  # Skip the header row
        student_data = None
        for row in data:
            if row[1] == student_id:  # Assuming stdID is in the second column
                student_data = row
                break

        if student_data is None:
            print(f"No student found with ID {student_id}.")
        else:
            print("\nStudent Details:")
            print("-" * 30)
            print(f"Student ID: {student_data[1]}")
            print(f"Name: {student_data[2]}")
            print(f"College: {student_data[3]}")
            print(f"Department: {student_data[4]}")
            print(f"Level: {student_data[5]}")
            print(f"Degree: {student_data[6]}")
            print(f"Major: {student_data[7]}")
            print(f"Minor: {student_data[8]}")
            print(f"Terms: {student_data[9]}")
        logPreviousRequest(student_id, "Student Details")
    except Exception as e:
        print(f"Error loading file: {e}")
    returnToMenu()

def statisticsFeature(student_id):
    # Set the file path for the student-specific CSV
    folder_path = r"C:\\Users\\admin\\JupyterJade"
    file_name = f"{student_id}.csv"  # Use the student ID for the file name
    file_path = os.path.join(folder_path, file_name)

    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"No transcript file found for Student ID {student_id}.")
        return

    try:
        # Read the CSV file
        data = np.genfromtxt(file_path, delimiter=',', dtype=None, encoding=None, skip_header=1)

        # Initialize variables to store data
        grades = []
        terms = []

        # Process each row and extract relevant information
        for row in data:
            grade = row[7]  # The Grade is in the 8th column (index 7)
            term = row[2]   # The Term is in the 3rd column (index 2)

            # Ensure that the grade is a valid numeric value
            try:
                grade = float(grade)  # Try converting the grade to a float
                grades.append(grade)
                terms.append(term)
            except ValueError:
                continue  # Skip invalid grades

        if not grades:
            print("No valid grades found in the transcript.")
            return

        # Calculate the overall average grade
        overall_avg = statistics.mean(grades)

        # Calculate average grade per term
        term_avg = {}
        for term, grade in zip(terms, grades):
            if term not in term_avg:
                term_avg[term] = []
            term_avg[term].append(grade)

        term_avg = {term: statistics.mean(grades) for term, grades in term_avg.items()}

        # Find the minimum and maximum grades
        min_grade = min(grades)
        max_grade = max(grades)

        # Output the statistics
        print("\nStatistics Summary:")
        print("-" * 30)
        print(f"Overall Average Grade: {overall_avg:.2f}")

        # Average grade per term
        for term, avg in term_avg.items():
            print(f"Average for Term {term}: {avg:.2f}")

        print(f"Minor Grade: {min_grade}")
        print(f"Maximum Grade: {max_grade}")

    except Exception as e:
        print(f"Error loading file: {e}")


def transcriptFeature(student_id, transcript_type):
    folder_path = r"C:\\Users\\admin\\JupyterJade"
    file_path = os.path.join(folder_path, f"std{student_id}FullTranscript.txt")

    # Debugging line: print the file path being constructed
    print(f"Looking for file at: {file_path}")

    if not os.path.exists(file_path):
        print(f"No transcript file found for Student ID {student_id}.")
        return

    try:
        with open(file_path, 'r') as file:
            transcript_data = file.readlines()

        if transcript_type == "Major":
            print("\nMajor Transcript:")
            print("-" * 30)
            for line in transcript_data:
                if "Major" in line:
                    print(line.strip())
        elif transcript_type == "Minor":
            print("\nMinor Transcript:")
            print("-" * 30)
            for line in transcript_data:
                if "Minor" in line:
                    print(line.strip())
        elif transcript_type == "Full":
            print("\nFull Transcript:")
            print("-" * 30)
            for line in transcript_data:
                print(line.strip())

        logPreviousRequest(student_id, f"{transcript_type} Transcript")
    except Exception as e:
        print(f"Error loading file: {e}")
    returnToMenu()

def previousRequests(student_id, session_requests):
    print("\nPrevious Requests:")
    print("-" * 30)
    if not session_requests:
        print("No requests made during this session.")
    else:
        print(f"{'Request':<30} {'Date and Time'}")
        print("-" * 30)
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        for request in session_requests:
            print(f"{request:<30} {current_time}")
    returnToMenu()

def logPreviousRequest(student_id, request_name):
    folder_path = r"C:\\Users\\admin\\JupyterJade"
    file_path = os.path.join(folder_path, f"std{student_id}PreviousRequests.txt")
    with open(file_path, 'a') as file:
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        file.write(f"{request_name} - {current_time}\n")

def terminateSystem(student_id, session_requests):
    print("\nTerminating the system...")
    logPreviousRequest(student_id, "System Termination")
    time.sleep(3)
    print("System has been terminated.")

def returnToMenu():
    print("\nReturning to menu...")
    time.sleep(2.5)  # Delay before returning to the menu

# Start the program
startFeature()


Welcome to the Student Management System


Please select your student level (U for Undergraduate, G for Graduate, B for Both):  g
Please select your graduate level (M for Master, D for Doctorate, B0 for Both):  n


Invalid input. Please choose from M, D, or B0.


Please select your graduate level (M for Master, D for Doctorate, B0 for Both):  m
Please enter your 9-digit student ID (e.g. 202006000):  202412011


Processing your selection...

Student Management System Menu
1. Student Details
2. Statistics
3. Transcript based on major courses
4. Transcript based on minor courses
5. Full transcript
6. Previous transcript requests
7. Select another student
8. Terminate the system


...


Previous Requests:
------------------------------
Request                        Date and Time
------------------------------
Student Details                2025-01-30 03:47:33
Statistics                     2025-01-30 03:47:33
Major Transcript               2025-01-30 03:47:33
Minor Transcript               2025-01-30 03:47:33
Full Transcript                2025-01-30 03:47:33

Returning to menu...

Student Management System Menu
1. Student Details
2. Statistics
3. Transcript based on major courses
4. Transcript based on minor courses
5. Full transcript
6. Previous transcript requests
7. Select another student
8. Terminate the system


Please select an option (1-8):  5


Processing your request for Full Transcript...
Looking for file at: C:\\Users\\admin\\JupyterJade\std202412011FullTranscript.txt

Full Transcript:
------------------------------
Level	Degree	Term	courseName	courseID	courseType	creditHours	Grade
G	BS1	1	Computer Engineering as a discipline	CMPE 101	Major	3	88
G	BS1	1	Programming Logic Design	CMPE 102	Major	6	86
G	BS1	1	Civil Welfare Trainning Ser 1	CWTS 001	Minor	2	83
G	BS1	1	Math in the Modern World	GEED 004	Minor	2	86
G	BS1	1	Scienc Technology and Society	GEED 007	Minor	2	89
G	BS1	2	Understanding the self	GEED 001	Minor	2	92
G	BS1	2	Basic Calculus	MATH 003	Major	4	94
G	BS1	2	Physical Activity Towards Health and Fitness 1	PATHFIT 1	Minor	2	96
G	BS1	2	General Physics	PHYS 001	Major	4	99
G	BS1	2	Computer Servicing	CMPE 103	Major	6	87
G	M1	1	Advanced Math	MATH 004	Minor	2	89
G	M1	1	Computer Programming Management	CMPE 104	Minor	2	85
G	M1	1	Basic Electronics	ECE 101	Major	4	84
G	M1	1	Basic Communication	ECE 102	Major	4	89
G	M1	1	Electronic

Please select an option (1-8):  8



Terminating the system...
System has been terminated.
